We have used a few resources to help us make this algorithm,

https://arpitbhayani.me/blogs/genetic-knapsack/

https://medium.com/koderunners/genetic-algorithm-part-3-knapsack-problem-b59035ddd1d6

https://stackoverflow.com/questions/10324015/fitness-proportionate-selection-roulette-wheel-selection-in-python

You will find that none of the code has been copied from anywhere, these resources were used to help us only.

In [25]:
from random import choice, randrange, random
import numpy as np

In [74]:
n = 10 # number of items
max_val = 4 # modular param for item val
max_wgt = 4 # param for item weight
wgt_limit = 20 # set limit for the sack (weight limit)
solutions_count = 500

In [75]:
items = {} # dict of items, each has weight and value (dict of lists)

val = [i for i in range(max_wgt + 1)]
wgt = [i for i in range(max_val + 1)]

def construct_pop(n, max_val, max_wgt):
    m = [i for i in range(max_val)] # list of possible values
    p = [i for i in range(max_wgt)] # list of possible weights

    item_val = [randrange(1, len(m) + 1) for i in range(n)] # random list of val for n items
    item_wgt = [randrange(1, len(p) + 1) for i in range(n)] # random list of weights ..

    item_vw = [(val, wgt) for val, wgt in zip(item_val, item_wgt)] # make tuple (x, y) of val and weight

    for idx, i in enumerate(item_vw): # get index and populate dict 'items'
        items[idx] = i # item i = (val, wgt)

    print("List of items (item number : (value, weight))")
    print(f"Number of items: {n}")
    print(f"Maximum weight: {max_wgt}")
    print(f"Maximum value: {max_val}")
    print(items) # check if everything went as intended (spoilers it did)

    return(items)

In [76]:
pop_0 = construct_pop(n, max_val = max_val, max_wgt = max_wgt)

List of items (item number : (value, weight))
Number of items: 10
Maximum weight: 4
Maximum value: 4
{0: (4, 3), 1: (3, 4), 2: (4, 4), 3: (1, 1), 4: (4, 3), 5: (4, 1), 6: (1, 4), 7: (1, 4), 8: (2, 3), 9: (1, 2)}


In [77]:
def bit_strings(sol, pop_size):
    bit_string_arr = []
    for i in range(sol):
        bit_string = [randrange(0, 2) for i in range(pop_size)] # 0 or 1, randomly, length is number of items
        bit_string_arr.append(bit_string)
    bit_string_arr = np.array(bit_string_arr)

    return bit_string_arr

solutions_initial = bit_strings(solutions_count, n)

In [95]:
def set_score(bit_strings, items, weight_limit):
    scores = []
    item_id = [item for item in items.keys()]
    item_val = [item[0] for item in items.values()]
    item_wgt = [item[1] for item in items.values()]

    for solution in bit_strings:
        total_val = 0
        total_wgt = 0
        for idx, bit in enumerate(solution):
            if bit == 1:
                total_val += item_val[idx]
                total_wgt += item_wgt[idx]
        if total_wgt > weight_limit:
            total_val = 0

        scores.append(total_val)

    return scores

scores = set_score(solutions_initial, pop_0, wgt_limit)

In [79]:
def regressed_safeguard(item_list, prev_solutions, post_solutions):
    post_solutions_copy = post_solutions.copy()
    prev_solutions_copy = prev_solutions.copy()

    prev_scores = set_score(prev_solutions_copy, item_list, wgt_limit)
    max_score_prev = max(prev_scores)
    prev_best_sol = prev_solutions[np.argmax(prev_scores)].copy()

    post_scores = set_score(post_solutions_copy, item_list, wgt_limit)
    max_score_post = max(post_scores)

    if max_score_post < max_score_prev:
        worst_index_post = np.argmin(post_scores)
        post_solutions_copy[worst_index_post] = prev_best_sol.copy()

    else:
        pass

    return post_solutions_copy

In [80]:
# https://stackoverflow.com/questions/10324015/fitness-proportionate-selection-roulette-wheel-selection-in-python
# the above link is what helped us implement the roulette wheel. how it works is simple, we generate some number between 0 and 1,
# then we iterate over all probabilities, add them until their sum exceeds the random number. This is the selected number.
# the fact we have also set bad solutions (weight exceeded) to 0 means they will NEVER be selected here, which is good

def roulette_prob_wheel(pop, solution_scores):
    total_score = np.sum(solution_scores) # sum of all the scores (used for probability wheel :) )
    probabilities = solution_scores / total_score # divides each score by the total, gets probability (proportion)

    survival = []
    for i in range(len(pop)):
        rand = random()
        prob_sum = 0 # we set this equal to 0, add until we exceed "rand"
    
        for idx, prob in enumerate(probabilities):
            prob_sum += prob # add prob to sum until exceed
            if prob_sum > rand:
                survival.append(pop[idx]) # appends good item to survivor list
                break

    return survival # returns 500, == original solution size, on the right track

selection = roulette_prob_wheel(solutions_initial, scores)

In [81]:
def crossover(item_list, sol_size, pop, cross_rate): # we will set a rate for crossovers, this way they do not happen every time.
    crossover_results = []
    crossover_results_safe = None
    parents = [(pop[i], pop[i + 1]) for i in range(0, len(pop), 2)] # gets parent pairs, done "2 by 2" -> 250 len
    cross_point = randrange(1, sol_size) # for each bit string, different crossover index (never 0 or 20)
    for idx, parent in enumerate(parents):
        rand = random() # between 0 and 1n
        child_1 = parents[idx][0]
        child_2 = parents[idx][1]
        if rand < cross_rate: # crossover only happens if below defined rate (args)
            child_1 = np.hstack((parent[0][:cross_point], parent[1][cross_point:])) # np.hstack just concats 2 vectors of 1 row
            child_2 = np.hstack((parent[1][:cross_point], parent[0][cross_point:]))
            crossover_results.append(child_1)
            crossover_results.append(child_2)
        else:
            crossover_results.append(child_1)
            crossover_results.append(child_2)
            
    crossover_results_safe = regressed_safeguard(item_list, pop, crossover_results)

    # crossover_results = np.concatenate(crossover_results)
    return crossover_results_safe

cross_results = crossover(pop_0, 20, selection, 0.5)

In [82]:
def mutate(item_list, sol_size, pop, mutate_rate):
    mutated_results = []
    mutated_results_safe = None
    for bit_string in pop:
        mutated_bit_string = bit_string.copy()  # Make a copy to avoid modifying the original
        for idxx, bit in enumerate(mutated_bit_string):
            rand = random()
            if rand < mutate_rate:
                mutated_bit_string[idxx] = abs(mutated_bit_string[idxx] - 1)
        mutated_results.append(mutated_bit_string)

    mutated_results_safe = regressed_safeguard(item_list, pop, mutated_results)

    return mutated_results_safe

mut = mutate(pop_0, solutions_count, cross_results, 0.1)

mut_score = set_score(mut, pop_0, wgt_limit)

print(max(mut_score))

print(mut[0])

22
[1 1 1 0 1 0 1 0 1 1]


In [83]:
def test(prev, post):
    print(id(prev[0]))
    print(id(post[0]))
    
test(cross_results, mut)

140556046237744
140556046612688


In [130]:
def genetic_algorithm(iterations, pop_size, cross_rate, mutate_rate, verbose=True):
    best_solution_idx = 0
    current_max_score = 0
    solutions_bit = bit_strings(pop_size, n)
    items = construct_pop(n, max_val, max_wgt)
    best_solution = None
    selected_items = None

    for i in range(iterations):
        iter = 0
        score_solutions = set_score(solutions_bit, items, wgt_limit)
        selected_solutions = roulette_prob_wheel(solutions_bit, score_solutions)
        if iter > 0:
            selected_solutions = regressed_safeguard(items, solutions_bit, selected_solutions)
        crossover_solutions = crossover(items, pop_size, selected_solutions, cross_rate)
        mutated_solutions = mutate(items, pop_size, crossover_solutions, mutate_rate)

        score_generation = set_score(mutated_solutions, items, wgt_limit)
        new_max_score = max(score_generation)
        max_score_idx = np.argmax(new_max_score)
        optim_solution = mutated_solutions[max_score_idx]

        if verbose:
            print(f"Iteration {i + 1}: current best solution : {optim_solution}, score : {new_max_score}")

        if new_max_score > current_max_score:
            current_max_score = new_max_score
            best_solution_idx = max_score_idx
            best_solution = optim_solution
            selected_items = [item for j, item in enumerate(items.values()) if best_solution[j] == 1]

        solutions_bit = mutated_solutions
        
        iter += 1

    return best_solution, current_max_score, selected_items

best_solution, best_score, selected_items = genetic_algorithm(100, 10, 0.4, 0.1, verbose = False)
print(f"\nBest Solution: {best_solution}")
weights = [item[1] for item in selected_items]
values = [item[0] for item in selected_items]
values_sum = np.sum(values)
weights_sum = np.sum(weights)
print("Selected Items:")
for item in selected_items:
    print(f"Item {item}: Value {item[0]}, Weight {item[1]}")

print(f"total value of solution : {values_sum}")
print(f"total weight of solution : {weights_sum}")


List of items (item number : (value, weight))
Number of items: 10
Maximum weight: 4
Maximum value: 4
{0: (3, 2), 1: (1, 1), 2: (3, 4), 3: (3, 2), 4: (1, 3), 5: (4, 4), 6: (1, 3), 7: (2, 4), 8: (2, 4), 9: (4, 1)}

Best Solution: [1 0 0 1 1 1 0 0 0 1]
Selected Items:
Item (3, 2): Value 3, Weight 2
Item (3, 2): Value 3, Weight 2
Item (1, 3): Value 1, Weight 3
Item (4, 4): Value 4, Weight 4
Item (4, 1): Value 4, Weight 1
total value of solution : 15
total weight of solution : 12
